## inner functions

In [1]:
def arithmetic(f):
    def add(a, b):
        return a + b
    def subtract(a, b):
        return a - b
    if f == 'add':
        return add
    elif f == 'subtract':
        return subtract
    else:
        return 'Yo, not known. '

In [2]:
add = arithmetic('add')
add(2, 4)

6

In [3]:
sub = arithmetic('subtract')
sub(2, 4)

-2

## scopes

In [5]:
def outer():
    x = 10
    def inner():
        x = 20
        print(x)
    inner()
    print(x)

outer()

20
10


In [7]:
def outer():
    x = 10
    def inner():
        nonlocal x
        x = 20
        print(x)
    inner()
    print(x)

outer()

20
20


In [10]:
x = 10
def f():
    x = 20
    print(x)
f()
print(x)

20
10


In [11]:
x = 10
def f():
    global x
    x = 20
    print(x)
f()
print(x)

20
20


In [16]:
x = 50

def one():
    x = 10

def two():
#     global x
    x = 30

def three():
    x = 100
    def four():
        x = 2
    four()
    print(x)
    
for func in [one, two, three]:
    func()
    print(x)

50
30
100
30


In [21]:
x = 50

def one():
    x = 10

def two():
    global x
    x = 30

def three():
    x = 100
    def four():
        nonlocal x
        x = 2
    four()
    print(x)
    
for func in [one, two, three]:
    func()
    print(x)

50
30
2
30


## Closures

Suppose that a function needs certain variables that are not present locally. The vars are then stored in closures so that there will be no errors

In [22]:
def outer():
    x = 10
    def inner():
        print(x)
    inner()

In [23]:
outer()

10


The above function call should have actually printed something like None or that the variable is not declared. But, it has reference to what x is from a scope just above it. So python actually stores this in closures. Let's inspect below. 

In [24]:
def outer():
    x = 10
    def inner():
        print(x)
    return inner

In [25]:
x = outer()

In [27]:
x.__closure__

(<cell at 0x1086e6858: int object at 0x1060ce590>,)

In [28]:
len(x.__closure__)

1

In [29]:
for i in x.__closure__:
    print(i)

<cell at 0x1086e6858: int object at 0x1060ce590>


In [30]:
for i in x.__closure__:
    print(i.cell_contents)

10


Aha! See, there is the value 10 that inner() needs

In the below example, we will show that, even if a function is modified after calling it once, the closure stores the old value and that it will always keep holding it

In [31]:
def my_special_function():
    print('You are running my_special_function()')

def get_new_func(func):
    def call_func():
        func()
    return call_func

new_func = get_new_func(my_special_function)
# new_func.__closure__[0].cell_contents()

# Rewrite my_special_function() here
def my_special_function():
    print('hello')
    
new_func()

You are running my_special_function()
You are running my_special_function()


What the above code means is that, first time when we call `get_new_func` and assign to `new_func`, the reference to `my_specian_function` which is nothing but the `print` statement gets stored in the `call_func`'s closure. Hence, even if we redefine `my_special_function`, the closure will always hold the old reference. 

### Important

> Q. What are decorators then?
A. A Decorator is a function that takes in a function and returns a modified version of the original function. Now, the inputs that the original function had could be modified or the functionality itself could be modified or both.

### Example

```python
@double_args
def add(a, b):
    return a + b

>> add(1,2)
6
```

The decorator `double_args` is a function that doubles the arguments i.e. 1*2=2 and 2*2=4 and then adds them, 2+4=6. Let's see how this is done or what the contents of the function/decorator `double_args` are.

In [39]:
def double_args(func):
    def inner(a, b):
        a = 2*a
        b = 2*b
        return func(a, b)
    return inner

In [40]:
@double_args
def add(a, b):
    return a + b

In [41]:
add(1,2)

6

In [47]:
# can also call this way
def double_args1(func):
    def inner(a, b):
        a = 2*a
        b = 2*b
        return func(a, b)
    return inner

def add1(a, b):
    return a + b

In [48]:
new_add = double_args1(add1)
new_add(1,5)

12

#### Some useful program below that can print the args passed in to a function

In [44]:
import inspect

def print_args(func):
    sig = inspect.signature(func)
    def wrapper(*args, **kwargs):
        bound = sig.bind(*args, **kwargs).arguments
        str_args = ', '.join(['{}={}'.format(k, v) for k, v in bound.items()])
        print('{} was called with {}'.format(func.__name__, str_args))
        return func(*args, **kwargs)
    return wrapper

@print_args
def my_function(a, b, c):
    print(a + b + c)

In [45]:
my_function(1,2,3)

my_function was called with a=1, b=2, c=3
6


# Decorators Advanced